# Flood (combined) — single-hazard runner
_Generated on 2025-09-14 15:44 UTC_

## Setup & Config

In [ ]:
# --- Environment & Imports ---
import os, sys, glob, json
from pathlib import Path

# Make sure both the repo root and a potential 'modules/' folder are importable
ROOT = Path.cwd()
if (ROOT / "modules").exists():
    sys.path.insert(0, str(ROOT / "modules"))
sys.path.insert(0, str(ROOT))

# Basic utilities
import matplotlib.pyplot as plt

# YAML config loader (fallback if modules.config_utils not found)
try:
    from modules.config_utils import load_config  # project helper if available
except Exception:
    import yaml
    def load_config(path):
        with open(path, "r", encoding="utf-8") as f:
            return yaml.safe_load(f)

# Helper to auto-detect a config file if not set
DEFAULT_CONFIG_CANDIDATES = ["config.yaml", "config.yml", "configs/config.yaml"]
for _cand in DEFAULT_CONFIG_CANDIDATES:
    if Path(_cand).exists():
        CONFIG_PATH = _cand
        break
else:
    # Put a placeholder if nothing found
    CONFIG_PATH = "config.yaml"  # adjust if needed
print(f"Using CONFIG_PATH = {CONFIG_PATH!r}")
CONFIG = load_config(CONFIG_PATH)
CONFIG

## Run combined flood processing

In [ ]:
# The combined flood usually needs hazard rasters + thresholds per subtype, and the AOI/infrastructure.
# This notebook assumes the module reads most parameters from CONFIG, but we also show a manual call.
from typing import Dict, Tuple, Optional

try:
    from modules.flood_combination import process_combined_flood
except Exception:
    from flood_combination import process_combined_flood

import geopandas as gpd

# AOI
aoi_path = CONFIG.get("aoi")
aoi_gdf = gpd.read_file(aoi_path) if aoi_path else None

# Optional: load all points/lines as single layers if your CONFIG provides them
points_path = (CONFIG.get("inputs", {}).get("points"))
lines_path  = (CONFIG.get("inputs", {}).get("lines"))
points_gdf = gpd.read_file(points_path) if points_path else None
lines_gdf  = gpd.read_file(lines_path) if lines_path else None

# Rasters dict if you want to override CONFIG: {subhazard_name: (path, threshold)}
# If your module already reads from CONFIG, you can keep this empty.
hazard_rasters: Dict[str, Tuple[str, Optional[float]]] = {}

results = process_combined_flood(
    config=CONFIG,
    hazard_rasters=hazard_rasters,
    aoi=aoi_gdf,
    points=points_gdf,
    lines=lines_gdf,
    sample_points_per_line=int(CONFIG.get("sample_points_per_line", 10))
)
print("Combined flood result keys:", list(results.keys()))

In [ ]:
# --- Browse outputs ---
out_dir = Path(CONFIG.get("output_dir", "output"))
out_dir.mkdir(parents=True, exist_ok=True)
maps = sorted(list(out_dir.rglob("*.png"))) + sorted(list(out_dir.rglob("*.jpg")))
rasters = sorted(list(out_dir.rglob("*.tif"))) + sorted(list(out_dir.rglob("*.tiff")))
vectors = sorted(list(out_dir.rglob("*.gpkg"))) + sorted(list(out_dir.rglob("*.geojson")))

print(f"Found {len(maps)} map images, {len(rasters)} rasters, {len(vectors)} vectors under {out_dir}")

# Show the latest map if available
if maps:
    from PIL import Image
    display(Image.open(maps[-1]))
else:
    print("No map images found yet.")